In [1]:
import pandas as pd
import numpy as np
from IPython.display import Image
np.set_printoptions(precision = 3)

In [2]:
data = pd.read_csv("jester-data-1.csv", header=None)

In [3]:
n_features = 2

#user_ratings = data.values
user_ratings = data.values[0:100]
latent_user_preferences = np.random.random((user_ratings.shape[0], n_features))
latent_item_features = np.random.random((user_ratings.shape[1],n_features))

In [4]:
latent_item_features

array([[0.777, 0.806],
       [0.533, 0.79 ],
       [0.689, 0.244],
       [0.091, 0.895],
       [0.954, 0.966],
       [0.781, 0.669],
       [0.355, 0.364],
       [0.816, 0.207],
       [0.513, 0.405],
       [0.015, 0.919],
       [0.017, 0.818],
       [0.584, 0.876],
       [0.317, 0.493],
       [0.464, 0.162],
       [0.185, 0.591],
       [0.045, 0.68 ],
       [0.093, 0.858],
       [0.974, 0.518],
       [0.187, 0.75 ],
       [0.123, 0.033],
       [0.376, 0.052],
       [0.796, 0.766],
       [0.824, 0.368],
       [0.159, 0.136],
       [0.961, 0.287],
       [0.516, 0.723],
       [0.234, 0.657],
       [0.221, 0.411],
       [0.401, 0.587],
       [0.778, 0.752],
       [0.594, 0.565],
       [0.131, 0.466],
       [0.664, 0.622],
       [0.703, 0.699],
       [0.031, 0.927],
       [0.527, 0.949],
       [0.358, 0.863],
       [0.782, 0.1  ],
       [0.288, 0.102],
       [0.548, 0.071],
       [0.882, 0.172],
       [0.159, 0.385],
       [0.648, 0.587],
       [0.4

In [5]:
latent_user_preferences

array([[0.905, 0.719],
       [0.921, 0.196],
       [0.973, 0.113],
       [0.417, 0.607],
       [0.263, 0.459],
       [0.698, 0.123],
       [0.108, 0.094],
       [0.525, 0.502],
       [0.933, 0.524],
       [0.931, 0.281],
       [0.039, 0.94 ],
       [0.52 , 0.267],
       [0.484, 0.365],
       [0.017, 0.626],
       [0.33 , 0.27 ],
       [0.411, 0.058],
       [0.915, 0.099],
       [0.786, 0.838],
       [0.147, 0.928],
       [0.282, 0.518],
       [0.615, 0.157],
       [0.475, 0.165],
       [0.656, 0.714],
       [0.932, 0.489],
       [0.109, 0.757],
       [0.145, 0.55 ],
       [0.565, 0.999],
       [0.167, 0.164],
       [0.432, 0.297],
       [0.334, 0.187],
       [0.246, 0.907],
       [0.29 , 0.252],
       [0.097, 0.241],
       [0.588, 0.587],
       [0.606, 0.239],
       [0.625, 0.653],
       [0.378, 0.346],
       [0.611, 0.366],
       [0.678, 0.539],
       [0.924, 0.636],
       [0.801, 0.852],
       [0.196, 0.029],
       [0.241, 0.864],
       [0.8

In [6]:
def predict_rating(user_id,item_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    return user_preference.dot(item_preference)

def train(user_id, item_id, rating,alpha = 0.0001):
    
    #print item_id
    prediction_rating = predict_rating(user_id, item_id)
    err =  ( prediction_rating- rating );
    #print err
    user_pref_values = latent_user_preferences[user_id][:]
    latent_user_preferences[user_id] -= alpha * err * latent_item_features[item_id]
    latent_item_features[item_id] -= alpha * err * user_pref_values
    return err
    


def sgd(iterations = 100000):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    for iteration in range(0,iterations):
        error = []
        for user_id in range(0,latent_user_preferences.shape[0]):
            for item_id in range(0,latent_item_features.shape[0]):
                rating = user_ratings[user_id][item_id]
                if(not np.isnan(rating)):
                    err = train(user_id,item_id,rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()   
        if(iteration%10000 == 0 ):
            print (mse)

In [7]:
sgd()

2652.420511214622


KeyboardInterrupt: 

In [8]:
predictions = latent_user_preferences.dot(latent_item_features.T)
predictions

array([[ 93.155,  10.637,  -7.095, ...,  88.805,  79.775,  84.314],
       [  2.019,   0.569,   0.129, ...,   2.217,   2.009,   2.159],
       [ 33.34 ,  72.08 ,  54.38 , ...,  90.687,  84.978,  97.042],
       ...,
       [ 28.335,  83.272,  64.568, ...,  96.064,  90.414, 104.033],
       [  5.508,  -2.281,  -2.845, ...,   2.741,   2.312,   2.136],
       [ 30.628,  75.437,  57.643, ...,  91.265,  85.686,  98.178]])

In [ ]:
values = [zip(user_ratings[i], predictions[i]) for i in range(0,predictions.shape[0])]
comparison_data = pd.DataFrame(values)
comparison_data.columns = data.columns
comparison_data.applymap(lambda x: "(%2.3f|%2.3f)"%(x[0],x[1]))

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,100
0,(74.000|93.155),(-7.820|10.637),(8.790|-7.095),(-9.660|-0.956),(-8.160|7.221),(-7.520|-1.240),(-8.500|-3.848),(-9.850|-0.919),(4.170|-0.609),(-8.980|0.236),...,(2.820|82.922),(99.000|82.151),(99.000|78.427),(99.000|88.487),(99.000|82.887),(99.000|84.099),(-5.630|78.835),(99.000|88.805),(99.000|79.775),(99.000|84.314)
1,(100.000|2.019),(4.080|0.569),(-0.290|0.129),(6.360|0.474),(4.370|0.665),(-2.380|-0.019),(-9.660|0.330),(-0.730|-0.007),(-5.340|-0.014),(8.880|0.511),...,(2.820|2.122),(-4.950|2.062),(-0.290|2.007),(7.860|2.166),(-0.190|2.112),(-2.140|2.132),(3.060|1.987),(0.340|2.217),(-4.320|2.009),(1.070|2.159)
2,(49.000|33.340),(99.000|72.080),(99.000|54.380),(99.000|99.333),(99.000|105.128),(9.030|1.197),(9.270|81.879),(9.030|2.295),(9.270|-0.480),(99.000|102.105),...,(99.000|95.175),(99.000|86.041),(99.000|89.964),(9.080|81.588),(99.000|93.259),(99.000|92.451),(99.000|84.293),(99.000|90.687),(99.000|84.978),(99.000|97.042)
3,(48.000|28.377),(99.000|75.835),(8.350|58.360),(99.000|105.692),(99.000|111.234),(1.800|1.367),(8.160|87.354),(-2.820|2.510),(6.210|-0.464),(99.000|108.550),...,(99.000|94.903),(99.000|85.250),(99.000|89.703),(0.530|80.033),(99.000|92.868),(99.000|91.917),(99.000|83.642),(99.000|89.683),(99.000|84.300),(99.000|96.782)
4,(91.000|15.878),(8.500|-1.595),(4.610|-4.050),(-4.170|-5.138),(-5.390|-3.887),(1.360|-0.293),(1.600|-4.811),(7.040|-0.288),(4.610|-0.091),(-0.440|-5.052),...,(5.190|10.865),(5.580|11.175),(4.270|10.278),(5.190|12.591),(5.730|10.954),(1.550|11.222),(3.110|10.638),(6.550|12.196),(1.800|10.781),(1.600|11.034)
5,(100.000|3.763),(-6.170|-7.758),(-3.540|-7.113),(0.440|-11.993),(-8.500|-12.006),(-7.090|-0.247),(-4.320|-10.140),(-8.690|-0.352),(-0.870|0.007),(-6.650|-12.226),...,(-3.540|-4.505),(-6.890|-3.474),(-0.680|-4.255),(-2.960|-2.412),(-2.180|-4.279),(-3.350|-4.081),(0.050|-3.541),(-9.080|-3.477),(-5.050|-3.545),(-3.450|-4.613)
6,(47.000|30.097),(99.000|75.561),(99.000|57.837),(99.000|104.988),(99.000|110.660),(8.590|1.333),(-9.850|86.709),(7.720|2.475),(8.790|-0.474),(99.000|107.851),...,(99.000|95.982),(99.000|86.376),(99.000|90.725),(99.000|81.322),(99.000|93.960),(2.330|93.040),(99.000|84.711),(99.000|90.917),(99.000|85.383),(99.000|97.878)
7,(100.000|4.365),(6.840|-5.433),(3.160|-5.278),(9.170|-8.705),(-6.210|-8.571),(-8.160|-0.201),(-1.700|-7.414),(9.270|-0.271),(1.410|-0.006),(-5.190|-8.853),...,(7.230|-1.805),(-1.120|-1.071),(-0.100|-1.702),(-5.680|-0.190),(-3.160|-1.641),(-3.350|-1.476),(2.140|-1.178),(-0.050|-0.956),(1.310|-1.164),(0.000|-1.858)
8,(100.000|4.176),(-3.790|-4.089),(-3.540|-4.124),(-9.420|-6.708),(-6.890|-6.533),(-8.740|-0.165),(-0.290|-5.740),(-5.290|-0.217),(-8.930|-0.010),(-7.860|-6.811),...,(4.370|-0.662),(-0.290|-0.104),(4.170|-0.623),(-0.290|0.629),(-0.290|-0.536),(-0.290|-0.399),(-0.290|-0.215),(-0.290|0.043),(-3.400|-0.197),(-4.950|-0.691)
9,(72.000|91.989),(3.010|21.096),(5.150|1.824),(5.150|14.521),(3.010|23.040),(6.410|-0.970),(5.150|9.117),(8.930|-0.500),(2.520|-0.642),(3.010|16.062),...,(99.000|92.047),(4.470|89.911),(99.000|87.049),(99.000|95.125),(99.000|91.717),(99.000|92.720),(99.000|86.549),(99.000|96.833),(99.000|87.532),(99.000|93.633)


In [ ]:
comparison_data

In [ ]:
comparison = comparison_data.to_latex()
text_file = open("comparison.txt", "w")
text_file.write(comparison)
text_file.close()